# Conduction avec polyMAC

In [ ]:
from trustutils import run

run.introduction("G.Jomee & L. Lecointre")

run.TRUST_parameters()

### Description: 

Test du solver de conduction avec les discrétisations polyMAC. Focus spécifique sur les conditions limites.

In [ ]:
from trustutils import run 
import os, sys
run.useMEDCoupling()
import medcoupling as mc

nb_meshes_max = {1,2} # niveaux de raffinement a tester 2 max

run.reset()

mesh = {
    # 2D meshes
    "Cartesian":      [ "VDF","PolyMAC","PolyMAC_P0","PolyMAC_P0P1NC"],
    "Triangles":      ["VEF","PolyMAC","PolyMAC_P0","PolyMAC_P0P1NC"]
}

formules = {
    "trigo":  ["cos(x)*sin(y)","2*cos(x)*sin(y)","(-sin(x)*sin(y))","cos(x)*cos(y)" ],
    "Himmelblau": ["((x^2+y-11)^2+(x+y^2-7)^2)","-(12*x^2+4*y-42+4*x+12*y^2-26)","(4*x^3+4*x*y-42*x+2*y^2-14)","(-22+2*x^2-26*y+4*x*y+4*y^3)" ]
}

run.initBuildDirectory()
origin, build_dir = os.getcwd(), run.BUILD_DIRECTORY
os.chdir(build_dir)
list_dis = []

for m, dis in mesh.items():
    for n in nb_meshes_max:
        for d in dis:
            for f, expr in formules.items():
                if d == "VEF":
                    loc = "FACES"
                else:
                    loc = "elem"
                os.system(f"mkdir -p {m}/{n}/{d}/{f}")
                run.addCaseFromTemplate("manufactured_solution.data",targetDirectory=f"{m}/{n}/{d}/{f}",targetData=f"{d}_{f}.data", dic={"dis_": f"{d}","func_":expr[0],"source_":expr[1],"dx_":expr[2],"dy_":expr[3],"loc_":loc})
            
os.chdir(origin)
run.printCases()
run.extractNRCases()

In [ ]:
from trustutils import plot
a = plot.Table(["function", "discretization", "mesh_type", "refinement", r"$L_1$(error)", r"$L_2$(error)", r"$L_{\infty}$(error)"])

# If you want to load a file similar to a text file, 
# you can use the method plot.loadText("direction_of_file.txt")
for m, dis in mesh.items():
    for d in dis:
        for f, expr in formules.items():
            for n in nb_meshes_max:
                data1 = plot.loadText(f"{m}/{n}/{d}/{f}/{d}_{f}_L1_NORM.son")
                data2 = plot.loadText(f"{m}/{n}/{d}/{f}/{d}_{f}_L2_NORM.son")
                data3 = plot.loadText(f"{m}/{n}/{d}/{f}/{d}_{f}_LINF_NORM.son")
                a.addLine([[f"{f}",f"{d}",f"{m}",f"{n}",data1[1][1],data2[1][1],data3[1][1]]],"")


display(a)

In [ ]:
from trustutils import visit

for m, dis in mesh.items():
    for d in dis:
        if d == "VEF": 
            loc = "FACES"
            dual = "_dual"
        else:
            loc = "ELEM"
            dual = ""
        for f, expr in formules.items():
            fig=visit.Show(f"{m}/1/{d}/{f}/{d}_{f}_ERROR.lata","Pseudocolor",f"TEMPERATURE_{loc}_dom{dual}",title=f"{m}/1/{d}/{f}",plotmesh=False,nY=2,nX=1)
            fig.add(f"{m}/1/{d}/{f}/{d}_{f}_ERROR.lata","Pseudocolor",f"ERREUR_{loc}_dom{dual}",plotmesh=False,xIndice=0,yIndice=1)
            fig.plot()

In [ ]:
run.tablePerf()